In [1]:
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
import os

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\orkunmanap\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
ps = PorterStemmer()
word_list_index = {}
word_list_counter = {}
counter = 0

In [4]:
for filename in os.listdir('TRAIN\\spam'):
    try:
        input_program = open("TRAIN\\spam\\"+filename, "r").read()

        # Replace email address with 'emailaddress'
        input_program = re.sub(r'[\w\.-]+@[\w\.-]+\.\w+', ' emailadress ', input_program)

        # Replace urls with 'webaddress'
        input_program = re.sub(r'[\w\.-]+@[\w\.-]+\.\w+', ' webadress ', input_program)

        # Replace money symbol with 'money-symbol'
        input_program = re.sub(r'£|\$', 'money-symbol', input_program)

        # Replace 10 digit phone number with 'phone-number'
        input_program = re.sub(".*?(\(?\d{3}\D{0,3}\d{3}\D{0,3}\d{4}).*?", ' phone-number ', input_program)

        # Replace normal number with 'number'
        input_program = re.sub('\d', ' number ', input_program)

        # remove punctuation
        input_program = re.sub(r'[^\w\d\s]', ' ', input_program)

        # remove whitespace between terms with single space
        input_program = re.sub(r'\s+', ' ', input_program)

        # remove leading and trailing whitespace
        input_program = re.sub(r'^\s+|\s*?$', ' ', input_program)

        # change words to lower case
        input_program = input_program.lower()

        words = word_tokenize(input_program)

        for w in words:
            lex = ps.stem(w)
            if lex not in word_list_index.keys():
                word_list_index[lex] = counter
                word_list_counter[lex] = 1
                counter = counter + 1
            else:
                word_list_counter[lex] =  word_list_counter[lex] +  1
    except UnicodeDecodeError:
        continue

In [5]:
#print("COUNTER")
#print(word_list_counter)

In [6]:
#print("INDEX")
#print(word_list_index)

In [7]:
word_list_index_ham = {}
word_list_counter_ham = {}

In [8]:
for filename in os.listdir('TRAIN\\ham'):
    #print(filename)
    try:
        input_program = open("TRAIN\\ham\\"+filename, "r").read()

        # Replace email address with 'emailaddress'
        input_program = re.sub(r'[\w\.-]+@[\w\.-]+\.\w+', ' emailadress ', input_program)

        # Replace urls with 'webaddress'
        input_program = re.sub(r'[\w\.-]+@[\w\.-]+\.\w+', ' webadress ', input_program)

        # Replace money symbol with 'money-symbol'
        input_program = re.sub(r'£|\$', 'money-symbol', input_program)

        # Replace 10 digit phone number with 'phone-number'
        input_program = re.sub(".*?(\(?\d{3}\D{0,3}\d{3}\D{0,3}\d{4}).*?", ' phone-number ', input_program)

        # Replace normal number with 'number'
        input_program = re.sub('\d', ' number ', input_program)

        # remove punctuation
        input_program = re.sub(r'[^\w\d\s]', ' ', input_program)

        # remove whitespace between terms with single space
        input_program = re.sub(r'\s+', ' ', input_program)

        # remove leading and trailing whitespace
        input_program = re.sub(r'^\s+|\s*?$', ' ', input_program)

        # change words to lower case
        input_program = input_program.lower()

        words = word_tokenize(input_program)

        for w in words:
            lex = ps.stem(w)
            if lex not in word_list_index_ham.keys():
                if lex in word_list_index.keys():
                    word_list_index_ham[lex] = word_list_index[lex]
                else:
                    word_list_index_ham[lex] = counter
                    counter = counter + 1
                word_list_counter_ham[lex] = 1
            else:
                word_list_counter_ham[lex] =  word_list_counter_ham[lex] +  1
    except UnicodeDecodeError:
        continue

In [9]:
#print("COUNTER")
#print(word_list_counter_ham)

In [10]:
#print("INDEX")
#print(word_list_index_ham)

In [11]:
import json

In [12]:
with open("FILES\\counter_spam.json", "w") as outfile: 
    json.dump(word_list_counter, outfile)

In [13]:
with open("FILES\\counter_ham.json", "w") as outfile: 
    json.dump(word_list_counter_ham, outfile)

In [14]:
with open("FILES\\index_spam.json", "w") as outfile: 
    json.dump(word_list_index, outfile)

In [15]:
with open("FILES\\index_ham.json", "w") as outfile: 
    json.dump(word_list_index_ham, outfile)

In [16]:
spamicity = {}
total_word_ham = 0
total_word_spam = 0

In [17]:
for i in word_list_counter_ham.keys():
    total_word_ham = total_word_ham + word_list_counter_ham[i]
total_word_ham

938948

In [18]:
for i in word_list_counter.keys():
    total_word_spam = total_word_spam + word_list_counter[i]
total_word_spam

886733

In [19]:
word_list_counter_all = word_list_counter | word_list_counter_ham

In [20]:
for i in word_list_counter_all.keys():
    if i not in word_list_counter_ham:
        spamicity[i] = 0.99
    elif i not in word_list_counter:
        spamicity[i] = 0.01
        #spamicity[i] = 0
    else:
        spamicity[i] = (word_list_counter[i] / total_word_spam) / (word_list_counter_ham[i] / total_word_ham + word_list_counter[i] / total_word_spam)

In [21]:
list(spamicity.keys())[0]

'fw'

In [22]:
spamicity['one']

0.35103649432341727

In [23]:
with open("FILES\\spamicity.json", "w") as outfile: 
    json.dump(spamicity, outfile)

In [24]:
import heapq
highest_spam = heapq.nlargest(8, spamicity, key=spamicity.get)

In [25]:
lowest_spam = heapq.nsmallest(8, spamicity, key=spamicity.get)

In [26]:
highest_spam

['xmln',
 'pad',
 'mortgag',
 'equiv',
 'symbolmoney',
 'ffff',
 'systemwork',
 'progid']

In [27]:
lowest_spam

['kde', 'kernel', 'lenni', 'postgresql', 'rpm', 'squeez', 'java', 'razor']

In [28]:
spamicity['xmln']

0.995480185459128

In [29]:
spamicity['kde']

0.0014117354171401372

In [30]:
average_spamicity = sum(spamicity.values()) / len(spamicity.values())

In [31]:
average_spamicity

0.6206642547871196

In [32]:
spam_mult = 1
spam_mult_minus1 = 1

print('Highest')
for i in highest_spam:
    print(spamicity[i])
    spam_mult = spam_mult * float(spamicity[i])
    spam_mult_minus1 = spam_mult_minus1 * float(1 - spamicity[i])

print('Lowest')
for j in lowest_spam:
    print(spamicity[j])
    spam_mult = spam_mult * float(spamicity[j])
    spam_mult_minus1 = spam_mult_minus1 * float(1 - spamicity[j])

p = spam_mult / (spam_mult + spam_mult_minus1)

Highest
0.995480185459128
0.9934105922975263
0.9916384524050126
0.9912511442918545
0.9904873937376096
0.99030842213989
0.99
0.99
Lowest
0.0014117354171401372
0.0018640403679879819
0.0024507879133528073
0.003517201272909299
0.0038707742709632397
0.0050893509980415
0.005301698682293238
0.0054008503318885535


In [33]:
p

0.000612424833780509

In [34]:
value = { k : word_list_index[k] for k in set(word_list_index) - set(word_list_index_ham) }

In [35]:
len(value)

36084

In [36]:
set1 = set(word_list_index.keys())
set2 = set(word_list_index_ham.keys())

len(set1 ^ set2)

55872

In [37]:
len(word_list_index)

45817